## Face_it Project

### EfficientNet Model with balanced Dataset

- Dataset: *merged_disgust_CLEAN.csv*
- Here we trained a deep learning model for facial emotion recognition using VGG16 with transfer learning.
- The model was fine-tuned on a merged dataset of facial expressions, including the disgust class from an extended source.
- After preprocessing and data augmentation, the network was trained for 25
epochs with early stopping and learning rate reduction strategies.

 This model achieved **___% validation accuracy**.

### Import all the libraries

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# ===================================================
# IMPORTS LIBRARIES
# ===================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.efficientnet import preprocess_input

from keras.models import Sequential
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.applications import EfficientNetB0
from keras.optimizers import Adam


from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator


print("✅ Libraries were imported!")

✅ Libraries were imported!


### Load Dataset & Data Preparation

In [7]:
# ====================================
#    LOAD CSV DATASET
# ====================================
csv_path = '/content/drive/MyDrive/Colab Notebooks/merged_disgust_clean.csv'
df = pd.read_csv(csv_path)

df.head()

,emotion,pixels
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [8]:
emotion_dict = { 0: "Angry",
                 1: "Disgust",
                 2: "Fear",
                 3: "Happy",
                 4: "Sad",
                 5: "Surprise",
                 6: "Neutral" }

#  DataFrame with emotion code, type, and count
emotion_counts = df['emotion'].value_counts().sort_index()

emotion_label = [emotion_dict[i] for i in emotion_counts.index]

summary = pd.DataFrame({ #'Emotion ID': emotion_counts.index,
                         'Emotion Label': emotion_label,
                        'Count': emotion_counts.values
                       })
summary

,Emotion Label,Count
0,Angry,4717
1,Disgust,4724
2,Fear,4802
3,Happy,8794
4,Sad,5912
5,Surprise,3248
6,Neutral,6046


### Image Preprocessing / Prep

In [9]:
# ====================================
# 4. IMAGE PROCESSING
# ====================================
pixels = df['pixels'].tolist()

# Convert grayscale to RGB
X_rgb = np.array([
    cv2.cvtColor(np.fromstring(p, sep=' ').reshape(48, 48).astype('uint8'), cv2.COLOR_GRAY2RGB)
    for p in pixels
], dtype='uint8')

# Resize in batches to 128×128 (lighter than 224×224)
def resize_images_in_batches(images, target_size=(128, 128), batch_size=1000):
    resized = []
    for i in range(0, len(images), batch_size):
        batch = images[i:i+batch_size]
        resized_batch = [cv2.resize(img, target_size) for img in batch]
        resized.extend(resized_batch)
    return np.array(resized, dtype='float32')

X_resized = resize_images_in_batches(X_rgb)

# Preprocess for EfficientNet
X_preprocessed = preprocess_input(X_resized)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(df['emotion'])
y = to_categorical(y_encoded)

In [ ]:
# ====================================
# 5. SPLIT DATA
# ====================================
X_train, X_temp, y_train, y_temp = train_test_split(
    X_preprocessed, y, test_size=0.2, random_state=42, stratify=y_encoded
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp.argmax(axis=1)
)

# ====================================
# 6. DATA AUGMENTATION
# ====================================
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True
)
train_datagen.fit(X_train)

In [ ]:
# ====================================
# 7. MODEL INITIALIZATION
# ====================================
def initialize_model():
    base_model = EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=(128, 128, 3)
    )
    base_model.trainable = False

    model = Sequential()
    model.add(Input(shape=(128, 128, 3)))
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(y.shape[1], activation='softmax'))  # Adjust to number of classes

    return model

# ====================================
# 8. COMPILE MODEL
# ====================================
def compile_model(model):
    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=0.0003),
        metrics=['accuracy']
    )
    return model

In [5]:
# ====================================
# 9. CALLBACKS
# ====================================
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=10,
    verbose=1,
    restore_best_weights=True
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

callbacks = [early_stopping, lr_scheduler]

In [ ]:
# ====================================
# 10. TRAINING
# ====================================
model = initialize_model()
model = compile_model(model)

history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    steps_per_epoch=int(len(X_train) / 32),
    epochs=30,
    callbacks=callbacks
)


In [ ]:
# ====================================
# 11. EVALUATION
# ====================================
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f"✅ Test Accuracy with EfficientNetB0: {test_acc * 100:.2f}%")

---

### Tryying on old version:

In [ ]:
# ===================================================
# IMAGE PROCESSING
# ===================================================

# Convert 'pixels' column to RGB numpy arrays
pixels = df['pixels'].tolist()

X = np.array([
    cv2.cvtColor( np.fromstring(p, sep=' ').reshape(48, 48).astype('uint8'),
                  cv2.COLOR_GRAY2RGB   )
    for p in pixels], dtype='float32')

# Resize to EfficientNet input size
X = np.array([cv2.resize(img, (224, 224)) for img in X])

# Preprocess for EfficientNet
X_preprocessed = preprocess_input(X)

# One-hot encode labels
y = to_categorical(df['emotion'].values, num_classes=7)

# Print shapes
print("X shape:", X.shape)  # Should be (num_samples, 48, 48, 3)
print("Y shape:", Y.shape)  # Should be (num_samples, 7)

### Split Data: Train, Validation & Test

In [ ]:
# ===================================================
# SPLIT: TRAIN, VALIDATION & TEST
# ===================================================

# Preprocess input for EfficientNet:
# it escales and normalizes images (pixel values between -1 and 1)
X_preprocessed = preprocess_input(X)

# 80% train + 20% the rest (temp):
X_train, X_temp, y_train, y_temp = train_test_split( X_preprocessed,
                                                       y,
                                                       test_size=0.2,
                                                       random_state=42,
                                                       stratify=df['emotion'] )
# From the rest: 10% test + 10% val:
X_val, X_test, y_val, y_test = train_test_split(X_temp,
                                                y_temp,
                                                test_size=0.5,
                                                random_state=42,
                                                stratify=y_temp.argmax(axis=1) )


print("Train shape:", X_train.shape, y_train.shape)
print("Validation shape:", X_val.shape, y_val.shape)
print("Test shape:", X_test.shape, y_test.shape)

In [ ]:
# ====================================
#  DATA AUGMENTATION
# ====================================

train_datagen = ImageDataGenerator( rotation_range=15,
                                    width_shift_range=0.15,
                                    height_shift_range=0.15,
                                    shear_range=0.15,
                                    zoom_range=0.15,
                                    horizontal_flip=True  )
train_datagen.fit(X_train)

### Pre-trained Model: EfficientNetB0

In [ ]:
# ===================================================
#  MODEL INITIALIZATION - EfficientNet
# ===================================================

def initialize_model():
    '''Instantiate and return an EfficientNetB0-based model for emotion classification'''

    # Load EfficientNetB0 base (without top classifier)
    base_model = EfficientNetB0( include_top=False,
                                 weights='imagenet',
                                 input_shape=(224, 224, 3)    )

    base_model.trainable = False  # Freeze base for transfer learning

    model = Sequential()
    model.add(Input(shape=(224, 224, 3)))
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(7, activation='softmax'))  # 7 classes (diff emotions) in total

    return model

In [ ]:
# ===================================================
#  COMPILER
# ===================================================

def compile_model(model):
    '''Compile the EfficientNet model for emotion classification'''

    model.compile(
                   optimizer=Adam(learning_rate=0.0003),
                   loss='categorical_crossentropy',
                   metrics=['accuracy']
                 )

    return model

In [ ]:
# ====================================
#  CALLBACKS
# ====================================

early_stopping = EarlyStopping( monitor='val_accuracy',
                                min_delta=0.00005,
                                patience=10,
                                verbose=1,
                                restore_best_weights=True )

lr_scheduler = ReduceLROnPlateau( monitor='val_accuracy',
                                  factor=0.5,
                                  patience=5,#
                                  min_lr=1e-7,
                                  verbose=1 )

callbacks = [early_stopping, lr_scheduler]

### Training the model

In [ ]:
# ====================================
#  TRAINING
# ====================================

model = initialize_model()

model = compile_model(model)

history = model.fit(
                    train_datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_val, y_val),
                    steps_per_epoch=int(len(X_train) / 32),
                    epochs=30,
                    callbacks=callbacks
                    )

___

In [ ]:
# ====================================
#  EVALUATION
# ====================================
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

print(f"✅ Test Accuracy with EfficientNetB0: {test_acc * 100:.2f}%")

In [ ]:
# ====================================
#  PLOTTING LEARNING CURVES
# ====================================
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))

# ACCURACY
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()


# LOSS
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# ====================================
#  CONFUSION MATRIX (adjust it!)
# ====================================

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Predict on validation set
y_pred_probs = model.predict(X_valid)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_valid, axis=1)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display with sklearn
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=list(emotion_dict.values()))
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Confusion Matrix")
plt.show()


### Key Insights